In [2]:
from src.graph import build_graph
from dotenv import load_dotenv
import logging
import pandas as pd
import json

load_dotenv()

graph = build_graph()

with open("prompts/system_prompt_short.txt", "r", encoding="utf-8") as f:
    system_message = f.read()

gaia_bench_1_test = pd.read_json('data/gaia_bench_1_test.jsonl', lines=True)
task = gaia_bench_1_test.iloc[8]

question = task.question
initial_state = {
    'system_message': system_message,
    'question': question,
    'file_name': task.file_name,
}

final_state = graph.invoke(initial_state)
final_answer = final_state.get("final_answer", None)

================================ System Message ================================

You are a general AI assistant. I will ask you a question. 
Report your thoughts, and finish your answer with the following template: 
FINAL ANSWER: [YOUR FINAL ANSWER]. 

YOUR FINAL ANSWER should be a number OR as few words as possible OR a comma separated list of numbers and/or strings. 
If you are asked for a number, don't use comma to write your number neither use units such as $ or percent sign unless specified otherwise. 
If you are asked for a string, don't use articles, neither abbreviations (e.g. for cities), and write the digits in plain text unless specified otherwise. 
If you are asked for a comma separated list, apply the above rules depending of whether the element to be put in the list is a number or a string.

================================ Human Message =================================

I'm making a grocery list for my mom, but she's a professor of botany and she's a real stickler when

In [ ]:
from langchain_community.retrievers import WikipediaRetriever
from langchain_community.document_loaders import WikipediaLoader

search_docs = WikipediaRetriever().invoke('Wikipedia:Featured articles/2016 November')


In [14]:
search_docs

[Document(metadata={'title': 'Wikipedia:Featured picture candidates/November-2016', 'summary': 'Please cut and paste new entries to the bottom of this page, creating a new monthly archive (by closing date) when necessary.', 'source': 'https://en.wikipedia.org/wiki/Wikipedia:Featured_picture_candidates/November-2016'}, page_content='Please cut and paste new entries to the bottom of this page, creating a new monthly archive (by closing date) when necessary.\n\n\n=== Palazzo Barberini (Rome) - Borromini\'s staircase ===\nVoting period is over. Please don\'t add any new votes. Voting period ends on 6 Nov 2016  at 07:51:27 (UTC)\n\nReason\nOne of Borromini\'s masterpiece\nArticles in which this image appears\nPalazzo Barberini\nFP category for this image\nPlaces/Interiors\nCreator\nLivioandronico2013\nSupport as nominator – LivioAndronico (talk) 07:51, 27 October 2016 (UTC)\nOppose Interior is too dark. Awkward angle. Where\'s the staircase? Étienne Dolet (talk) 17:44, 27 October 2016 (UTC)

In [ ]:
final_state['last_ai_message']

{'messages': [SystemMessage(content='You are a helpful assistant tasked with answering questions using a set of tools.\nYour final answer for my question must strictly follow this format:\nFINAL ANSWER: [ANSWER]\nYour answer should only start with "FINAL ANSWER: ", followed by the answer. \nWrite the answer in that exact format.', additional_kwargs={}, response_metadata={}, id='79c02a47-fcae-4396-85e0-b58cb45edb82'),
  HumanMessage(content='.rewsna eht sa "tfel" drow eht fo etisoppo eht etirw ,ecnetnes siht dnatsrednu uoy fI', additional_kwargs={}, response_metadata={}, id='03576908-163d-4593-b481-70fa359ee5a4'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yy7nqt400o7afpws73jxi7vl', 'function': {'arguments': '{"query":".rewsna eht sa \\"tfel\\" drow eht fo etisoppo eht etirw ,ecnetnes siht dnatsrednu uoy fI"}', 'name': 'reverse_string'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1342, 'prompt_t

In [6]:
def validate_answer():
    """Validate the final answer."""
  
    pattern = 'FINAL ANSWER: '

    i = final_state['last_ai_message'].find(pattern)
    final_answer = final_state['last_ai_message'][i + len(pattern):]
    print(final_answer)
    return {"final_answer": final_answer}
validate_answer()

right


{'final_answer': 'right'}

In [3]:
from langchain_community.retrievers import WikipediaRetriever

query = 'Mercedes Sosa discography'

search_docs = WikipediaRetriever(load_max_docs=5).invoke(query)

In [7]:
print(search_docs[0].page_content)

Haydée Mercedes Sosa (9 July 1935 – 4 October 2009) was an Argentine singer who was popular throughout Latin America and many countries outside the region. With her roots in Argentine folk music, Sosa became one of the preeminent exponents of El nuevo cancionero. She gave voice to songs written by many Latin American songwriters. Her music made people hail her as the "voice of the voiceless ones". She was often called "the conscience of Latin America".
Sosa performed in venues such as the Lincoln Center in New York City, the Théâtre Mogador in Paris, the Sistine Chapel in Vatican City, as well as sold-out shows in New York's Carnegie Hall and the Roman Colosseum during her final decade of life. Her career spanned four decades and she was the recipient of six Latin Grammy awards (2000, 2003, 2004, 2006, 2009, 2011), including a Latin Grammy Lifetime Achievement Award in 2004 and two posthumous Latin Grammy Award for Best Folk Album in 2009 and 2011. She won the Premio Gardel in 2000, th

In [9]:
from langchain_community.tools import DuckDuckGoSearchResults

search_docs_dds = DuckDuckGoSearchResults(max_results=3, output_format='list').invoke(query)


/Users/olyandrevn/Git/hf_agents/gaia_agent/.venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


In [10]:
search_docs_dds

[{'snippet': 'Over MercedesForum.nl/be Hét Mercedes-Benz forum voor de echte liefhebber. Je vindt hier het laatste Mercedes nieuws, opgeloste problemen, technische gegevens en handleidingen, te koop …',
  'title': 'MercedesForum.nl/be - Alles over Mercedes-Benz',
  'link': 'https://www.mercedesforum.nl/'},
 {'snippet': 'Jul 25, 2023 · I was told a few months ago (during an MOT) that MB had an ongoing rear subframe recall issue going on. More recently I was buying parts from the local MB dealership (excellent …',
  'title': 'Rear subframe replacement recall - MB policy - Mercedes-Benz …',
  'link': 'https://forums.mercedesclub.org.uk/index.php?threads/rear-subframe-replacement-recall-mb-policy.198069/'},
 {'snippet': 'Jan 13, 2015 · Hi guys, Does anyone know what this fault code means from STAR? Ive looked all over tinternet and cant find any answers!? P068500 The output for relay circuit 87 (combustion …',
  'title': "The output for relay circuit 87 fault? - Mercedes-Benz Owners' …",
 

In [13]:
import wikipedia

url = 'https://en.wikipedia.org/wiki/Mercedes_Sosa'

print(wikipedia.page('Mercedes Sosa Discography').content)

Haydée Mercedes Sosa (9 July 1935 – 4 October 2009) was an Argentine singer who was popular throughout Latin America and many countries outside the region. With her roots in Argentine folk music, Sosa became one of the preeminent exponents of El nuevo cancionero. She gave voice to songs written by many Latin American songwriters. Her music made people hail her as the "voice of the voiceless ones". She was often called "the conscience of Latin America".
Sosa performed in venues such as the Lincoln Center in New York City, the Théâtre Mogador in Paris, the Sistine Chapel in Vatican City, as well as sold-out shows in New York's Carnegie Hall and the Roman Colosseum during her final decade of life. Her career spanned four decades and she was the recipient of six Latin Grammy awards (2000, 2003, 2004, 2006, 2009, 2011), including a Latin Grammy Lifetime Achievement Award in 2004 and two posthumous Latin Grammy Award for Best Folk Album in 2009 and 2011. She won the Premio Gardel in 2000, th

In [7]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/Wikipedia:Featured_articles_promoted_in_2016'

response = requests.get(url)
response_text = response.text
soup = BeautifulSoup(response_text, "html.parser")

In [8]:
print(' '.join(soup.get_text().split()))

Wikipedia:Featured articles promoted in 2016 - Wikipedia Jump to content Main menu Main menu move to sidebar hide Navigation Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us Contribute HelpLearn to editCommunity portalRecent changesUpload fileSpecial pages Search Search Appearance Donate Create account Log in Personal tools Donate Create account Log in Pages for logged out editors learn more ContributionsTalk Contents move to sidebar hide (Top) 1 Promoted in December 2016 2 Promoted in November 2016 3 Promoted in October 2016 4 Promoted in September 2016 5 Promoted in August 2016 6 Promoted in July 2016 7 Promoted in June 2016 8 Promoted in May 2016 9 Promoted in April 2016 10 Promoted in March 2016 11 Promoted in February 2016 12 Promoted in January 2016 Toggle the table of contents Wikipedia:Featured articles promoted in 2016 2 languages Русский粵語 Edit links Project pageTalk English ReadEditView history Tools Tools move to sidebar hide Actions ReadEditView histo

In [9]:
soup.get_text()

'\n\n\n\nWikipedia:Featured articles promoted in 2016 - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nPromoted in December 2016\n\n\n\n\n\n\n\n\n2\nPromoted in November 2016\n\n\n\n\n\n\n\n\n3\nPromoted in October 2016\

In [10]:
print(soup.get_text())





Wikipedia:Featured articles promoted in 2016 - Wikipedia


























Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload fileSpecial pages



















Search











Search






















Appearance
















Donate

Create account

Log in








Personal tools





Donate Create account Log in





		Pages for logged out editors learn more



ContributionsTalk




























Contents
move to sidebar
hide




(Top)





1
Promoted in December 2016








2
Promoted in November 2016








3
Promoted in October 2016








4
Promoted in September 2016








5
Promoted in August 2016








6
Promoted in July 2016








7
Promoted in June 2016








8
Promoted in May 2016








9
Promoted in April 2016








10
Promoted in March 2016





